<a href="https://colab.research.google.com/github/anurag1210/ML_ZoomCamp_Homework_2024/blob/main/Homework_9_Serverless.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing all the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [1]:
from google.colab import files
uploaded = files.upload()

Saving model_2024_hairstyle.keras to model_2024_hairstyle.keras


In [2]:
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model('model_2024_hairstyle.keras')

# Convert the model to TF-Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted TF-Lite model
with open('model_2024_hairstyle.tflite', 'wb') as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpwaxqo1um'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  133905113387552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133905113391952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133905113417152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133905113422080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133905113384032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133905113385440: TensorSpec(shape=(), dtype=tf.resource, name=None)


Question 1
Now convert this model from Keras to TF-Lite format.

What's the size of the converted model?

In [3]:
import os

# Check the size of the TF-Lite model
tflite_file_size = os.path.getsize('model_2024_hairstyle.tflite') / (1024 * 1024)  # Size in MB
print(f"TF-Lite Model Size: {tflite_file_size:.2f} MB")

TF-Lite Model Size: 76.58 MB


Question 2
To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?

In [9]:
pip install pillow

In [10]:
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="model_2024_hairstyle.tflite")
interpreter.allocate_tensors()

In [11]:
# Get the output tensor details
output_details = interpreter.get_output_details()

# Print the output index
print(f"Output Index: {output_details[0]['index']}")

Output Index: 13


Q3. Now we need to turn the image into numpy array and pre-process it.

Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

In [12]:
from io import BytesIO
from urllib import request
from PIL import Image
import numpy as np

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

# Download and resize the image
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'
img = download_image(url)
target_size = (224, 224)  # Replace with your model's input size
img = prepare_image(img, target_size)

In [13]:
# Convert the image to a NumPy array
img_array = np.array(img)

# Pre-process the image: normalize pixel values to [0, 1]
img_array = img_array / 255.0

In [14]:
# Get the R channel value of the first pixel
first_pixel_r_value = img_array[0, 0, 0]
print(f"First pixel R channel value: {first_pixel_r_value}")

First pixel R channel value: 0.23921568627450981


Question 4
Now let's apply this model to this image. What's the output of the model?

In [19]:
import numpy as np
import tensorflow as tf
from io import BytesIO
from urllib import request
from PIL import Image

# Step 1: Download the image from the URL
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

# Step 2: Pre-process the image
def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')  # Ensure the image is in RGB format
    img = img.resize(target_size, Image.NEAREST)  # Resize to model's input size
    return img

# Step 3: Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="model_2024_hairstyle.tflite")
interpreter.allocate_tensors()

# Step 4: Get input and output details from the model
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Step 5: Download and prepare the image
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'
img = download_image(url)

# Check the model's expected input shape
print(f"Expected input shape: {input_details[0]['shape']}")

# Set the target size based on the model's expected input (e.g., 200x200)
target_size = (200, 200)  # Adjust this if needed based on the shape output
img = prepare_image(img, target_size)

# Convert the image to a numpy array and normalize the pixel values
img_array = np.array(img) / 255.0  # Normalize to [0, 1]
input_array = np.expand_dims(img_array, axis=0).astype(np.float32)  # Add batch dimension and ensure type is float32

# Step 6: Run the model on the input image
interpreter.set_tensor(input_details[0]['index'], input_array)
interpreter.invoke()

# Step 7: Get and print the model output
output = interpreter.get_tensor(output_details[0]['index'])
print(f"Model output: {output[0]}")

Expected input shape: [  1 200 200   3]
Model output: [0.8937741]
